# PyODB Performance Tests


In [1]:
import cProfile
from datetime import datetime
from pathlib import Path
from tqdm import tqdm
from time import sleep, time

from src.pyodb.pyodb import PyODB, PyODBCache
from test.test_models.complex_models import ComplexBasic, ComplexContainer, ComplexMulti
from test.test_models.primitive_models import PrimitiveBasic, PrimitiveContainer

In [ ]:
def test_insert_performance():
    pyodb = PyODB(2)
    pyodb.add_type(ComplexBasic)
    pyodb.save(ComplexContainer())
    pyodb.add_type(ComplexMulti)

    for _ in tqdm(range(10)):
        pyodb.save_multiple([ComplexBasic() for _ in range(100)])
        pyodb.save_multiple([ComplexContainer() for _ in range(10)])
        pyodb.save_multiple([ComplexMulti() for _ in range(100)])

        pyodb.delete(ComplexBasic).gt(random_number = 0).commit()
        pyodb.remove_type(ComplexContainer)
        pyodb.add_type(ComplexContainer)
    del pyodb


base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_insert_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
cProfile.run("test_insert_performance()", filepath.as_posix())

In [ ]:
def insert_base_data():
    pyodb = PyODB(persistent=True)
    pyodb.add_type(ComplexBasic)
    pyodb.add_type(ComplexContainer)
    pyodb.add_type(ComplexMulti)

    for _ in tqdm(range(10), desc="Inserting base data"):
        pyodb.save_multiple([ComplexBasic() for _ in range(100)])
        pyodb.save_multiple([ComplexContainer() for _ in range(10)])
        pyodb.save_multiple([ComplexMulti() for _ in range(100)])


def test_select_performance():
    pyodb = PyODB()
    for _ in tqdm(range(100), desc="Testing PrimitiveBasic"):
        pyodb.select(PrimitiveBasic).all()
    for _ in tqdm(range(100), desc="Testing PrimitiveContainer"):
        pyodb.select(PrimitiveContainer).all()
    for _ in tqdm(range(100), desc="Testing ComplexBasic"):
        pyodb.select(ComplexBasic).all()
    for _ in tqdm(range(100), desc="Testing ComplexContainer"):
        pyodb.select(ComplexContainer).all()


base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_select_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
insert_base_data()
cProfile.run("test_select_performance()", filepath.as_posix())

In [2]:
cache = PyODBCache(PyODB(1))
cache.add_cache("test", lambda x: [PrimitiveBasic() for _ in range(x)], PrimitiveBasic, 1)
print(cache.get_data("test", 100))

start = time()
print(cache["test"])
print(time() - start)

sleep(1)
print(cache.get_data("test", 1))
del cache

[PrimitiveBasic: 394, -116.02881257641839, '{zMeC#Y2P" *u8Y=oRm=k8Xrd0lTxwa3RQcgjqF#r)eG$lq.]'McK}w{3+rqF1', False, PrimitiveBasic: -989, None, '#x(d°5kTL !&5yp3!#'CQiQ5o0JHP)w"qkv_rhq~°PL^p,KV1(a6$XOG7#M>kFcP8a#BtX)D§&0`)_~o&+b1&;,Tf7cR`<kD/R', False, PrimitiveBasic: -839, -146.08947603938293, '°tyuZ{!Hey$E5>q´IM^oanM}L& u~wt<]'&6kXU{Ar{}YMyY´H3ISE0zc3:Oc}-E*´tK,vd', True, PrimitiveBasic: -347, 205.3658764479098, '+[S1O>D~*_K23aAA~=v;;´EOf^xn#=~UeclcQn`T7q=QCL-:(/\<,{´R52.eSW&}RKS', False, PrimitiveBasic: -18, None, 'b3W(:#;TLNQf!I4kH77115QhB)Ef Gb#i;0\N2Uni6*= `wERfIz8j8RMKeK7M;v}°hSLlgS=M!<E{4Kqi,&bO+QIxzz5x#°Fp', True, PrimitiveBasic: -569, -614.7838639367342, 'rfe_5x+XO/c\EaD"+jR>j! m.zY61j&FSPg]<4$d´aEfN0{:H071NS5G^A(\'^ SpHG:=^O6[\7jmkn(HG§k)X0QVYrf3-S', True, PrimitiveBasic: -44, 995.4384213965186, ']qu{2d]F^< BxD-l8&Zkd3bV5:)*°UCT[[p^TYQhM', True, PrimitiveBasic: 126, None, '$)&BBAZvL´Zv=FoG^5jn/SqBNhvb:Zh|u">.H#YV{nV=-6v_{'oF', True, PrimitiveBasic: -733, -918.5993889587107, 